In [39]:
from HelperFunctions import *
import time
import itertools

In [40]:
def generate_sigma(p): # TODO: Needs to be finished in helper functions, for now pasted in here
    ''' 
    Generates a list 'sigma' of length p, where sigma is the variance of each feature vector dimension i.e. x_i ~ N(0, sigma_p)

    input: p (int) represents the number of features

    returns: sigma (list of floats)
        EXAMPLE:    
            n = 100
            p = 5 
            grid_dim = 5 
            sigma = [0.1, 0.2, 0.3, 0.4, 0.5]
            noise = 0.25
            degree = 3

            X, C = generate_data(n, p, grid_dim, sigma, noise, degree)
            A,b = CreateShortestPathConstraints(grid_dim)
            B1 = DirectSolution(A, b, X, C)
    ''' 
    #not done continue later
    sigma = []
    for i in range(p): 
        num = abs(np.random.normal())
        sigma.append(num)

    return sigma 

In [41]:
def problem_size_experiment(params, noise, degree):
    
    ''' 
    Runs the direct and SGD solvers with given input paraemters
    
    input: 
        dict{str:list} params: dictionary of parameter values to experiment with. Must specify 'n', 'p', and 'grid_size'
        float noise: multiplicative noise term applied to cost vector, sampled from uniform distribution in [1-noise, 1+noise]
        int degree: polynomial degree of generated cost vector. When degree=1, expected value of c is linear in x. Degree > 1 controls the amount of model misspecification.

    returns: dict{str:list} with experimental results including: runtime, SPO loss, and SPO plus loss for both direct and SGD solvers
    ''' 
    
    # Variable definitions
    experimental_results = {}
    
    direct_runtime = []
    SGD_runtime = []
    
    SPO_loss_direct = []
    SPO_loss_SGD = []
    
    SPO_plus_loss_direct= []
    SPO_plus_loss_SGD = []
    
    # Record the values we're varying (will make plotting easier)
    experimental_results['params'] = params
    
    # Define static sigma if not varying p
    if len(params['p']) == 1:
        sigma_const = generate_sigma(params['p'][0])
    
    # For each parameter combo solve the problem instance and record results
    for n,p,grid_dim in itertools.product(params['n'], params['p'], params['grid_dim']):

        # Use static sigma if not varying p in experiment 
        if len(params['p']) == 1:
            sigma = sigma_const
        else:   
            sigma = generate_sigma(p)
                    
        # Generate the dataset
        X, C = generate_data(n, p, grid_dim, sigma, noise, degree)
        
        # Create shortest path contraints
        A,b = CreateShortestPathConstraints(grid_dim)
        
        # Run the direct solution and record the time
        start_direct = time.time()
        B_direct=DirectSolution(A,b, X, C)
        end_direct = time.time() - start_direct
        direct_runtime.append(end_direct)
    
        # Run the SGD solution and record the time
        start_sgd = time.time()
        B_SGD=GradientDescentSolution(A,b, X, C, batch_size=10,epsilon = 0.001) 
        end_sgd = time.time() - start_sgd
        SGD_runtime.append(end_sgd)
        
        # Record losses
        solver = ShortestPathSolver(A,b)
        SPO_loss_direct.append(SPOLoss(solver, X, C, B_direct))
        SPO_loss_SGD.append(SPOLoss(solver, X, C, B_SGD))
        SPO_plus_loss_direct.append(SPOplusLoss(solver, X, C, B_direct))
        SPO_plus_loss_SGD.append(SPOplusLoss(solver, X, C, B_SGD))

    # Add lists to results dictionary
    experimental_results['direct_runtime'] = direct_runtime
    experimental_results['SGD_runtime'] = SGD_runtime
    experimental_results['SPO_loss_direct'] = SPO_loss_direct
    experimental_results['SPO_loss_SGD'] = SPO_loss_SGD
    experimental_results['SPO_plus_loss_direct'] = SPO_plus_loss_direct
    experimental_results['SPO_plus_loss_SGD'] = SPO_plus_loss_SGD

    return experimental_results

In [45]:
params = {"n": [10], "p": [1,5], "grid_dim": [5]}
noise = 0.25
degree = 3

experiment = problem_size_experiment(params, noise, degree)
experiment

Converged after 1122 steps
Converged after 138 steps


{'params': {'n': [10], 'p': [1, 5], 'grid_dim': [5]},
 'direct_runtime': [0.06683588027954102, 0.04893207550048828],
 'SGD_runtime': [21.965145111083984, 2.564466953277588],
 'SPO_loss_direct': [10.38072666766127, 0.0],
 'SPO_loss_SGD': [10.178097044490665, 0.8227327121033341],
 'SPO_plus_loss_direct': [25.708982734850252, 1.3500311979441903e-14],
 'SPO_plus_loss_SGD': [29.66968405712704, 3.7034035246452612]}

In [43]:
params = {"n": [10,100], "p": [5], "grid_dim": [5]}
noise = 0.25
degree = 3

experiment = problem_size_experiment(params, noise, degree)
experiment

Converged after 657 steps
Converged after 1625 steps


{'params': {'n': [10, 100], 'p': [5], 'grid_dim': [5]},
 'direct_runtime': [0.057869911193847656, 3.8285560607910156],
 'SGD_runtime': [13.416093111038208, 32.62833595275879],
 'SPO_loss_direct': [0.0, 4.09275298431221],
 'SPO_loss_SGD': [0.8853072887576673, 5.035403172038496],
 'SPO_plus_loss_direct': [5.684341886080802e-15, 16.612236910446413],
 'SPO_plus_loss_SGD': [9.344590268864398, 21.203627828040684]}

In [44]:
params = {"n": [10], "p": [5], "grid_dim": [5,10]}
noise = 0.25
degree = 3

experiment = problem_size_experiment(params, noise, degree)
experiment

Converged after 159 steps
Converged after 635 steps


{'params': {'n': [10], 'p': [5], 'grid_dim': [5, 10]},
 'direct_runtime': [0.08979034423828125, 0.39901185035705566],
 'SGD_runtime': [3.3987178802490234, 15.803815841674805],
 'SPO_loss_direct': [0.0, 0.0],
 'SPO_loss_SGD': [0.6520711862785447, 6.788852434139335],
 'SPO_plus_loss_direct': [-8.526512829121202e-15, 3.694822225952521e-14],
 'SPO_plus_loss_SGD': [6.818277755469735, 19.944584000638905]}